In [ ]:
from datasets import load_dataset

ds_raw = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")
ds = load_dataset("Salesforce/wikitext", "wikitext-2-v1")

test-00000-of-00001.parquet:   0%|          | 0.00/685k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.07M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/618k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [33]:
import transformers
name = 'roberta-base'
tokenizer = transformers.AutoTokenizer.from_pretrained(name,device='cpu')
pipe = transformers.pipeline('fill-mask', model=name, tokenizer=tokenizer)

Device set to use cpu


In [49]:
tokenized = tokenizer(['Townsend recorded a Noisescapes demo and sent copies to various record labels .'])
tokenized['input_ids'][0][-2] = tokenizer.mask_token_id
print(tokenizer.decode(tokenized['input_ids'][0]))
pipe(tokenizer.decode(tokenized['input_ids'][0]))

<s>Townsend recorded a Noisescapes demo and sent copies to various record labels<mask></s>


[{'score': 0.8297228813171387,
  'token': 4,
  'token_str': '.',
  'sequence': 'Townsend recorded a Noisescapes demo and sent copies to various record labels.'},
 {'score': 0.15780211985111237,
  'token': 35,
  'token_str': ':',
  'sequence': 'Townsend recorded a Noisescapes demo and sent copies to various record labels:'},
 {'score': 0.0013476395979523659,
  'token': 72,
  'token_str': '."',
  'sequence': 'Townsend recorded a Noisescapes demo and sent copies to various record labels."'},
 {'score': 0.0010444007348269224,
  'token': 4832,
  'token_str': ' :',
  'sequence': 'Townsend recorded a Noisescapes demo and sent copies to various record labels :'},
 {'score': 0.000980676501058042,
  'token': 734,
  'token_str': '...',
  'sequence': 'Townsend recorded a Noisescapes demo and sent copies to various record labels...'}]

In [31]:
print(ds['train']['text'][1500:1510])
print(ds_raw['train']['text'][1500:1510])

[" Townsend recorded a Noisescapes demo and sent copies to various record labels . <unk> Records responded to Townsend with a record deal and Townsend began work on what was to be the first Noisescapes album , Promise . Shortly afterward , the label introduced him to musician Steve Vai . Impressed with Townsend 's vocal work , Vai offered him the role of the lead vocalist on his new album Sex and Religion . After recording Sex and Religion , Townsend accompanied Vai on a world tour in support of the album . Townsend soon landed a second touring gig , this time with the opening band of Vai 's tour , the Wildhearts . He played live with the band throughout half of 1994 in Europe , and appeared as a guest musician on their single <unk> . <unk> , the band 's frontman , remained close friends with Townsend , later co @-@ writing several songs on Infinity and the <unk> + 4 <unk> EP . \n", ' While on tour with the Wildhearts , Townsend formed a short @-@ lived thrash metal project with Metall

In [ ]:
inp = ds_raw['train']['text'][1500]

#wikitext-2 was described in https://arxiv.org/pdf/1609.07843
# it was made with the moses tokenizer, plus a few minor enhancements, such as "8,600 -> 8 @,@ 600"

def normalize_text(str):
    str = str.replace('\n',' ')
    #removing double white space
    last = str
    first = True 
    while(not(last == str) or first):
        last = str
        str = str.replace("  "," ")
        first = False 
    #fixing adjoining @-@'s 
    joiners = ["-",","]
    for joiner in joiners:
        str = str.replace(" @"+joiner+"@ ",joiner)
    #fixing white space introduced by punctuation 
    puncts = ['.',',','?','!',':',';']
    for punct in puncts:
        str = str.replace(" " + punct, punct)
    #fixing clitics
    clitics = ["'s","'re","'ve","'ll","n't","'d","'m"]
    for clitic in clitics:
            str = str.replace(" "+clitic, clitic)
    return str 

import sacremoses 
moses_detokenizer = sacremoses.MosesDetokenizer(lang='en')

def normalize_text_moses(str):
    output = moses_detokenizer.detokenize(str.split(' '), return_str=True)
    output = output.replace(" @,@ ",",") # this was a custom comma introduced by the wikitext-2 dataset for numbers.
    return output

out = normalize_text(inp)
print(inp)
print(out)
out_moses = normalize_text_moses(inp)
print(out_moses)


 Townsend recorded a Noisescapes demo and sent copies to various record labels . Relativity Records responded to Townsend with a record deal and Townsend began work on what was to be the first Noisescapes album , Promise . Shortly afterward , the label introduced him to musician Steve Vai . Impressed with Townsend 's vocal work , Vai offered him the role of the lead vocalist on his new album Sex and Religion . After recording Sex and Religion , Townsend accompanied Vai on a world tour in support of the album . Townsend soon landed a second touring gig , this time with the opening band of Vai 's tour , the Wildhearts . He played live with the band throughout half of 1994 in Europe , and appeared as a guest musician on their single Urge . Ginger , the band 's frontman , remained close friends with Townsend , later co @-@ writing several songs on Infinity and the Christeen + 4 Demos EP . 

 Townsend recorded a Noisescapes demo and sent copies to various record labels. Relativity Records r

'8-600.'

In [ ]:
import nltk.tokenize
nltk_tokenizer = nltk.tokenize.TreebankWordTokenizer()
out = nltk_tokenizer.tokenize("I wanna watch something")
nltk_detokenizer = nltk.tokenize.treebank.TreebankWordDetokenizer()
nltk_detokenizer.detokenize(out)

'I wanna watch something'

In [128]:
#let's see if we can find all of the article headers. 
import re
pattern = r'^[^=]*=[^=]*=[^=]*\n$'
source = ds_raw['train']['text']
matches = [(i,string) for i,string in enumerate(source) if re.match(pattern,string)]
matches = [(i,string) for i,string in matches if i > 0 and i < len(source)-2 and source[i-1] == "" and source[i+1] == ""]
article_ranges = [(matches[i][0],matches[i+1][0]) for i in range(len(matches)-1)]
print(article_ranges)
for m in matches:
    print(source[m[0]-1:m[0]+2])

[(1, 51), (51, 133), (133, 271), (271, 287), (287, 316), (316, 406), (406, 433), (433, 464), (464, 480), (480, 507), (507, 557), (557, 610), (610, 628), (628, 748), (748, 799), (799, 913), (913, 944), (944, 976), (976, 1038), (1038, 1107), (1107, 1157), (1157, 1244), (1244, 1274), (1274, 1380), (1380, 1401), (1401, 1487), (1487, 1582), (1582, 1633), (1633, 1648), (1648, 1690), (1690, 1710), (1710, 1856), (1856, 1960), (1960, 1988), (1988, 2038), (2038, 2134), (2134, 2156), (2156, 2187), (2187, 2270), (2270, 2347), (2347, 2378), (2378, 2487), (2487, 2555), (2555, 2665), (2665, 2680), (2680, 2712), (2712, 2733), (2733, 2820), (2820, 2932), (2932, 3128), (3128, 3259), (3259, 3299), (3299, 3345), (3345, 3443), (3443, 3475), (3475, 3571), (3571, 3640), (3640, 3717), (3717, 3733), (3733, 3790), (3790, 3820), (3820, 3926), (3926, 3941), (3941, 4015), (4015, 4150), (4150, 4199), (4199, 4228), (4228, 4439), (4439, 4557), (4557, 4573), (4573, 4787), (4787, 4818), (4818, 4836), (4836, 5057), (505

In [ ]:
import re
pattern = r'^[^=]*=[^=]*=[^=]*\n$'

# Test the regex pattern
test_string = " = Valkyria Chronicles III = \n"
match = re.match(pattern, test_string)
print(bool(match))  # Output: True if the string matches the pattern, otherwise False

True


In [111]:
match = re.match(pattern, " =  Valkyria Chronicles III  = \n")
print(match)
print(bool(match))

<re.Match object; span=(0, 32), match=' =  Valkyria Chronicles III  = \n'>
True
